# Classical BERT

In [1]:
!pip install pandas scikit-learn torch transformers

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# step 2
import pandas as pd
import ast

# Load preprocessed CSV
df = pd.read_csv(r"E:\Sem 7 Project\Datasets\Preprocessed Datasets\output_part_1.csv")

# Convert list-like columns to strings if needed
df['question_text'] = df['question_text'].apply(ast.literal_eval).apply(lambda x: ' '.join(x))
df['topic'] = df['topic'].apply(ast.literal_eval).apply(lambda x: x[0] if len(x) > 0 else '')

print(df[['question_text', 'topic']].head())

                                       question_text      topic
0  organ choos concern eye stage three point desp...    quantum
1  certainli turn marriag consid environ school p...     climat
2                       marriag best film dream wish  psycholog
3  guess soon hair compani letter fund mission me...    histori
4  know say congress view wear amount money succe...     machin


In [46]:
# step 3
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['topic'])
num_labels = len(le.classes_)
print(df[['topic', 'label']].drop_duplicates().sort_values('label'))

               topic  label
6           artifici      0
28        blockchain      1
10            career      2
1             climat      3
5         cybersecur      4
39              data      5
17             digit      6
9               educ      7
22  entrepreneurship      8
7        environment      9
69            health     10
3            histori     11
4             machin     12
31            person     13
2          psycholog     14
0            quantum     15
20             robot     16
18          sociolog     17
8              space     18
14               web     19


In [4]:
# step 4
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['topic'])
num_labels = len(le.classes_)
print(df[['topic', 'label']].drop_duplicates().sort_values('label'))

               topic  label
6           artifici      0
28        blockchain      1
10            career      2
1             climat      3
5         cybersecur      4
39              data      5
17             digit      6
9               educ      7
22  entrepreneurship      8
7        environment      9
69            health     10
3            histori     11
4             machin     12
31            person     13
2          psycholog     14
0            quantum     15
20             robot     16
18          sociolog     17
8              space     18
14               web     19


In [5]:
# step 5
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(texts, max_length=64):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'train_texts' is not defined

In [51]:
# step 6
import torch

class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BertDataset(train_encodings, train_labels)
val_dataset = BertDataset(val_encodings, val_labels)

In [52]:
# step 7
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./N-Results/classical_bert',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,3.043700
20,3.076600
30,2.974200
40,2.953300
50,2.889000
60,2.770500
70,2.706800
80,2.600500
90,2.483800
100,2.531600


TrainOutput(global_step=200, training_loss=2.3929232215881346, metrics={'train_runtime': 1262.5559, 'train_samples_per_second': 1.267, 'train_steps_per_second': 0.158, 'total_flos': 52630715596800.0, 'train_loss': 2.3929232215881346, 'epoch': 10.0})

In [53]:
# train time
import time

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,1.688900
20,1.557200
30,1.391800
40,1.385700
50,1.161800
60,1.043800
70,0.837600
80,0.772000
90,0.623500
100,0.634200


In [54]:
# step 8
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Get predictions
preds = trainer.predict(val_dataset)
y_true = val_labels.values
y_pred = preds.predictions.argmax(axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=le.classes_))

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 0.05
Precision: 0.03571428571428571
Recall: 0.05
F1-Score: 0.04

Classification Report:
                   precision    recall  f1-score   support

        artifici       0.33      0.33      0.33         3
      blockchain       0.00      0.00      0.00         3
          career       0.00      0.00      0.00         2
          climat       0.00      0.00      0.00         2
      cybersecur       0.00      0.00      0.00         2
            data       0.00      0.00      0.00         3
           digit       0.00      0.00      0.00         3
            educ       0.00      0.00      0.00         1
entrepreneurship       0.00      0.00      0.00         2
     environment       0.00      0.00      0.00         2
          health       0.00      0.00      0.00         1
         histori       0.00      0.00      0.00         1
          machin       0.00      0.00      0.00         1
          person       0.14      0.33      0.20         3
       psycholog       0.00   

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

In [56]:
# step 9
# test_texts = ["marriag best film dream wish"]
test_texts = ['guess soon hair compani letter fund mission medic import tradit everyon']
test_encodings = tokenize(test_texts)
model.eval()
with torch.no_grad():
    outputs = model(**{k: v for k, v in test_encodings.items()})
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = probs.argmax(dim=1).item()
    pred_topic = le.inverse_transform([pred_label])[0]
    print("Predicted topic:", pred_topic)

Predicted topic: histori


# DistillBert Model

In [17]:
!pip install pandas scikit-learn torch transformers

Defaulting to user installation because normal site-packages is not writeable


In [6]:
# step 2 : load & preprocess data
import pandas as pd
import ast

# Load preprocessed CSV
df = pd.read_csv(r"E:\\Sem 7 Project\\Datasets\\Preprocessed Datasets\\output_part_1.csv")

# Convert list-like columns to strings if needed
df['question_text'] = df['question_text'].apply(ast.literal_eval).apply(lambda x: ' '.join(x))
df['topic'] = df['topic'].apply(ast.literal_eval).apply(lambda x: x[0] if len(x) > 0 else '')

print(df[['question_text', 'topic']].head())
# Expected: First 5 rows of question_text and topic columns

                                       question_text      topic
0  organ choos concern eye stage three point desp...    quantum
1  certainli turn marriag consid environ school p...     climat
2                       marriag best film dream wish  psycholog
3  guess soon hair compani letter fund mission me...    histori
4  know say congress view wear amount money succe...     machin


In [7]:
# step 3
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['topic'])
num_labels = len(le.classes_)
print(df[['topic', 'label']].drop_duplicates().sort_values('label'))
# Expected: Unique topics and their encoded labels

               topic  label
6           artifici      0
28        blockchain      1
10            career      2
1             climat      3
5         cybersecur      4
39              data      5
17             digit      6
9               educ      7
22  entrepreneurship      8
7        environment      9
69            health     10
3            histori     11
4             machin     12
31            person     13
2          psycholog     14
0            quantum     15
20             robot     16
18          sociolog     17
8              space     18
14               web     19


In [8]:
# step 4 - train test split
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)
print(len(train_texts), len(val_texts))
# Expected: Number of training and validation samples

160 40


In [13]:
# step 5
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)
print(train_encodings.keys())
# Expected: dict_keys(['input_ids', 'attention_mask'])

KeysView({'input_ids': tensor([[  101, 17935,  3775,  ...,     0,     0,     0],
        [  101,  3813,  2529,  ...,     0,     0,     0],
        [  101,  2025,  2594,  ...,     0,     0,     0],
        ...,
        [  101,  2530,  3519,  ...,     0,     0,     0],
        [  101,  2345,  5770,  ...,     0,     0,     0],
        [  101,  3693,  3095,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})


In [14]:
# step 6 - dataset class preparation
import torch

class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BertDataset(train_encodings, train_labels)
val_dataset = BertDataset(val_encodings, val_labels)
print(len(train_dataset), len(val_dataset))
# Expected: Number of samples in train and validation datasets

160 40


In [15]:
# step 7 - model training
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import time

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./N-Results/results_distilbert',
    num_train_epochs=12,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./N-Results/logs_distilbert',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time (seconds): {training_time:.2f}")
print(f"Training Time (minutes): {training_time/60:.2f}")
# Expected: Training time in seconds and minutes

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,3.015500
20,3.018600
30,2.950400
40,2.911000
50,2.815500
60,2.788500
70,2.623300
80,2.547700
90,2.326300
100,2.326200


Training Time (seconds): 3011.14
Training Time (minutes): 50.19


In [16]:
# step 8 - model evaluation
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Get predictions
preds = trainer.predict(val_dataset)
y_true = val_labels.values
y_pred = preds.predictions.argmax(axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
cm = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=le.classes_)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:\n", report)
# Expected: ML metrics and confusion matrix

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 0.025
Precision: 0.009375
Recall: 0.025
F1-Score: 0.013636363636363636
Confusion Matrix:
[[0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Classification Report:
                

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

In [17]:
# step 9
test_texts = ["marriag best film dream wish"]
test_encodings = tokenize(test_texts)
model.eval()
with torch.no_grad():
    outputs = model(**{k: v for k, v in test_encodings.items()})
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = probs.argmax(dim=1).item()
    pred_topic = le.inverse_transform([pred_label])[0]
    print("Predicted topic:", pred_topic)
# Expected: Predicted topic for the input text

Predicted topic: machin


# Roberta Model

In [1]:
!pip install pandas scikit-learn torch transformers

Defaulting to user installation because normal site-packages is not writeable


In [18]:
# step 2
import pandas as pd
import ast

# Load preprocessed CSV
df = pd.read_csv(r"E:\\Sem 7 Project\\Datasets\\Preprocessed Datasets\\output_part_1.csv")

# Convert list-like columns to strings if needed
df['question_text'] = df['question_text'].apply(ast.literal_eval).apply(lambda x: ' '.join(x))
df['topic'] = df['topic'].apply(ast.literal_eval).apply(lambda x: x[0] if len(x) > 0 else '')

print(df[['question_text', 'topic']].head())
# Expected: First 5 rows of question_text and topic columns

                                       question_text      topic
0  organ choos concern eye stage three point desp...    quantum
1  certainli turn marriag consid environ school p...     climat
2                       marriag best film dream wish  psycholog
3  guess soon hair compani letter fund mission me...    histori
4  know say congress view wear amount money succe...     machin


In [19]:
# step 3
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['topic'])
num_labels = len(le.classes_)
print(df[['topic', 'label']].drop_duplicates().sort_values('label'))
# Expected: Unique topics and their encoded labels

               topic  label
6           artifici      0
28        blockchain      1
10            career      2
1             climat      3
5         cybersecur      4
39              data      5
17             digit      6
9               educ      7
22  entrepreneurship      8
7        environment      9
69            health     10
3            histori     11
4             machin     12
31            person     13
2          psycholog     14
0            quantum     15
20             robot     16
18          sociolog     17
8              space     18
14               web     19


In [20]:
# step 4 - train test split
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)
print(len(train_texts), len(val_texts))
# Expected: Number of training and validation samples

160 40


In [21]:
# step 5
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def tokenize(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)
print(train_encodings.keys())
# Expected: dict_keys(['input_ids', 'attention_mask'])

KeysView({'input_ids': tensor([[    0, 28878,   118,  ...,     1,     1,     1],
        [    0,   506,  9856,  ...,     1,     1,     1],
        [    0,  3654,   636,  ...,     1,     1,     1],
        ...,
        [    0, 16507, 12442,  ...,     1,     1,     1],
        [    0,  6156,  1796,  ...,     1,     1,     1],
        [    0, 26960,   813,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})


In [22]:
# step 6
import torch

class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BertDataset(train_encodings, train_labels)
val_dataset = BertDataset(val_encodings, val_labels)
print(len(train_dataset), len(val_dataset))
# Expected: Number of samples in train and validation datasets

160 40


In [23]:
# step 7
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
import time

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./N-Results/results_roberta',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./N-Results/logs_roberta',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time (seconds): {training_time:.2f}")
print(f"Training Time (minutes): {training_time/60:.2f}")
# Expected: Training time in seconds and minutes

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,3.020100
20,3.015300
30,2.994000
40,3.003500
50,2.978900
60,3.003600
70,2.973500
80,2.990100
90,2.963800
100,3.001100


Training Time (seconds): 3901.69
Training Time (minutes): 65.03


In [24]:
# step 8
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Get predictions
preds = trainer.predict(val_dataset)
y_true = val_labels.values
y_pred = preds.predictions.argmax(axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
cm = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=le.classes_)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:\n", report)
# Expected: ML metrics and confusion matrix

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 0.1
Precision: 0.05863095238095238
Recall: 0.1
F1-Score: 0.06999999999999999
Confusion Matrix:
[[0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 2 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Classification Report:
          

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

In [26]:
# step 9

# test_texts = ["marriag best film dream wish"]
# test_encodings = tokenize(test_texts)
test_texts = ['guess soon hair compani letter fund mission medic import tradit everyon']
test_encodings = tokenize(test_texts)
model.eval()
with torch.no_grad():
    outputs = model(**{k: v for k, v in test_encodings.items()})
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = probs.argmax(dim=1).item()
    pred_topic = le.inverse_transform([pred_label])[0]
    print("Predicted topic:", pred_topic)
# Expected: Predicted topic for the input text

Predicted topic: digit


# Albert Model

In [1]:
!pip install pandas scikit-learn torch transformers

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# step 2
import pandas as pd
import ast

# Load preprocessed CSV
df = pd.read_csv(r"E:\\Sem 7 Project\\Datasets\\Preprocessed Datasets\\output_part_1.csv")

# Convert list-like columns to strings if needed
df['question_text'] = df['question_text'].apply(ast.literal_eval).apply(lambda x: ' '.join(x))
df['topic'] = df['topic'].apply(ast.literal_eval).apply(lambda x: x[0] if len(x) > 0 else '')

print(df[['question_text', 'topic']].head())
# Expected: First 5 rows of question_text and topic columns

                                       question_text      topic
0  organ choos concern eye stage three point desp...    quantum
1  certainli turn marriag consid environ school p...     climat
2                       marriag best film dream wish  psycholog
3  guess soon hair compani letter fund mission me...    histori
4  know say congress view wear amount money succe...     machin


In [2]:
# step 3
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['topic'])
num_labels = len(le.classes_)
print(df[['topic', 'label']].drop_duplicates().sort_values('label'))
# Expected: Unique topics and their encoded labels

               topic  label
6           artifici      0
28        blockchain      1
10            career      2
1             climat      3
5         cybersecur      4
39              data      5
17             digit      6
9               educ      7
22  entrepreneurship      8
7        environment      9
69            health     10
3            histori     11
4             machin     12
31            person     13
2          psycholog     14
0            quantum     15
20             robot     16
18          sociolog     17
8              space     18
14               web     19


In [3]:
# step 4
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)
print(len(train_texts), len(val_texts))
# Expected: Number of training and validation samples

160 40


In [4]:
# step 5
from transformers import AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

def tokenize(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)
print(train_encodings.keys())
# Expected: dict_keys(['input_ids', 'attention_mask'])

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeysView({'input_ids': tensor([[   2, 7821, 1388,  ...,    0,    0,    0],
        [   2, 1904,  585,  ...,    0,    0,    0],
        [   2,   52,  596,  ...,    0,    0,    0],
        ...,
        [   2,  650, 1656,  ...,    0,    0,    0],
        [   2,  426, 4234,  ...,    0,    0,    0],
        [   2, 1865, 1138,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})


In [5]:
# step 6
import torch

class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BertDataset(train_encodings, train_labels)
val_dataset = BertDataset(val_encodings, val_labels)
print(len(train_dataset), len(val_dataset))
# Expected: Number of samples in train and validation datasets

160 40


In [6]:
# step 7
from transformers import AlbertForSequenceClassification, Trainer, TrainingArguments
import time

model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./N-Results/results_albert',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./N-Results/logs_albert',
    logging_steps=10,
    # evaluation_strategy='epoch',
    # save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time (seconds): {training_time:.2f}")
print(f"Training Time (minutes): {training_time/60:.2f}")
# Expected: Training time in seconds and minutes

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,3.104700
20,3.118600
30,3.086500
40,3.055000
50,3.028900
60,3.080800
70,3.039500
80,3.051200
90,3.007500
100,3.047900


Training Time (seconds): 2422.72
Training Time (minutes): 40.38


In [7]:
# step 8
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Get predictions
preds = trainer.predict(val_dataset)
y_true = val_labels.values
y_pred = preds.predictions.argmax(axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
cm = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=le.classes_)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:\n", report)
# Expected: ML metrics and confusion matrix

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 0.025
Precision: 0.01
Recall: 0.025
F1-Score: 0.014285714285714285
Confusion Matrix:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]]

Classification Report:
                   p

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

In [8]:
# step 9
# test_texts = ["marriag best film dream wish"]
test_texts = ['guess soon hair compani letter fund mission medic import tradit everyon']
test_encodings = tokenize(test_texts)
model.eval()
with torch.no_grad():
    outputs = model(**{k: v for k, v in test_encodings.items()})
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = probs.argmax(dim=1).item()
    pred_topic = le.inverse_transform([pred_label])[0]
    print("Predicted topic:", pred_topic)
# Expected: Predicted topic for the input text

Predicted topic: person


# MobileBert Bert

In [12]:
!pip install pandas scikit-learn torch transformers

Defaulting to user installation because normal site-packages is not writeable


In [9]:
# step 1
import pandas as pd
import ast

# Load preprocessed CSV
df = pd.read_csv(r"E:\\Sem 7 Project\\Datasets\\Preprocessed Datasets\\output_part_2.csv")

# Convert list-like columns to strings if needed
df['question_text'] = df['question_text'].apply(ast.literal_eval).apply(lambda x: ' '.join(x))
df['topic'] = df['topic'].apply(ast.literal_eval).apply(lambda x: x[0] if len(x) > 0 else '')

print(df[['question_text', 'topic']].head())
# Expected: First 5 rows of question_text and topic columns

                                       question_text             topic
0  keep somebodi maintain thing everybodi pay con...            climat
1  audienc somebodi medic countri ball morn deter...             space
2  person summer compani agre easi social caus ma...  entrepreneurship
3  mention hair blue especi health compar eight r...          artifici
4  arm success confer answer effort nation moment...          artifici


In [10]:
# step 2
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['topic'])
num_labels = len(le.classes_)
print(df[['topic', 'label']].drop_duplicates().sort_values('label'))
# Expected: Unique topics and their encoded labels

               topic  label
3           artifici      0
22        blockchain      1
16            career      2
0             climat      3
49        cybersecur      4
81              data      5
7              digit      6
21              educ      7
2   entrepreneurship      8
51       environment      9
9             health     10
10           histori     11
5             machin     12
20            person     13
6          psycholog     14
26           quantum     15
42             robot     16
56          sociolog     17
1              space     18
8                web     19


In [11]:
# step 3
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)
print(len(train_texts), len(val_texts))
# Expected: Number of training and validation samples

240 60


In [12]:
# step 4
from transformers import MobileBertTokenizer

tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

def tokenize(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)
print(train_encodings.keys())
# Expected: dict_keys(['input_ids', 'attention_mask'])

KeysView({'input_ids': tensor([[  101,  5323,  2397,  ...,     0,     0,     0],
        [  101,  2711,  2621,  ...,     0,     0,     0],
        [  101,  9358,  9496,  ...,     0,     0,     0],
        ...,
        [  101, 10346,  2319,  ...,     0,     0,     0],
        [  101,  7487,  2128,  ...,     0,     0,     0],
        [  101,  2028,  4929,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})


In [13]:
# step 5
import torch

class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BertDataset(train_encodings, train_labels)
val_dataset = BertDataset(val_encodings, val_labels)
print(len(train_dataset), len(val_dataset))
# Expected: Number of samples in train and validation datasets

240 60


In [14]:
# step 6
from transformers import MobileBertForSequenceClassification, Trainer, TrainingArguments
import time

model = MobileBertForSequenceClassification.from_pretrained('google/mobilebert-uncased', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./N-Results/results_mobilebert',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./N-Results/logs_mobilebert',
    logging_steps=10,
    # evaluation_strategy='epoch',
    # save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time (seconds): {training_time:.2f}")
print(f"Training Time (minutes): {training_time/60:.2f}")
# Expected: Training time in seconds and minutes

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,4130552.800000
20,51243.484400
30,219.016600
40,3.393600
50,3.143400
60,3.237400
70,2.857900
80,2.934600
90,2.984000
100,2.618500


Training Time (seconds): 1228.76
Training Time (minutes): 20.48


In [15]:
# step 7
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# Get predictions
preds = trainer.predict(val_dataset)
y_true = val_labels.values
y_pred = preds.predictions.argmax(axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
cm = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred, target_names=le.classes_)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:\n", report)
# Expected: ML metrics and confusion matrix

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 0.06666666666666667
Precision: 0.08166666666666668
Recall: 0.06666666666666667
F1-Score: 0.06841269841269841
Confusion Matrix:
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 2 0 1 1 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

C

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

In [16]:
# step 8

# test_texts = ['guess soon hair compani letter fund mission medic import tradit everyon']
test_texts = ["marriag best film dream wish"]
test_encodings = tokenize(test_texts)
model.eval()
with torch.no_grad():
    outputs = model(**{k: v for k, v in test_encodings.items()})
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = probs.argmax(dim=1).item()
    pred_topic = le.inverse_transform([pred_label])[0]
    print("Predicted topic:", pred_topic)
# Expected: Predicted topic for the input text

Predicted topic: person


# Electra Bert

In [26]:
!pip install pandas scikit-learn torch transformers

Defaulting to user installation because normal site-packages is not writeable


In [17]:
# step 1
import pandas as pd
import ast

# Load your preprocessed CSV
df = pd.read_csv(r"E:\\Sem 7 Project\\Datasets\\Preprocessed Datasets\\output_part_1.csv")  # Change path if needed

# Convert list-like columns to strings if needed
if isinstance(df['question_text'].iloc[0], str) and df['question_text'].iloc[0].startswith("["):
    df['question_text'] = df['question_text'].apply(ast.literal_eval).apply(lambda x: ' '.join(x))
if isinstance(df['topic'].iloc[0], str) and df['topic'].iloc[0].startswith("["):
    df['topic'] = df['topic'].apply(ast.literal_eval).apply(lambda x: x[0] if len(x) > 0 else '')

print(df[['question_text', 'topic']].head())

                                       question_text      topic
0  organ choos concern eye stage three point desp...    quantum
1  certainli turn marriag consid environ school p...     climat
2                       marriag best film dream wish  psycholog
3  guess soon hair compani letter fund mission me...    histori
4  know say congress view wear amount money succe...     machin


In [18]:
# step 2
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['topic'])
num_labels = len(le.classes_)
print(df[['topic', 'label']].drop_duplicates().sort_values('label'))

               topic  label
6           artifici      0
28        blockchain      1
10            career      2
1             climat      3
5         cybersecur      4
39              data      5
17             digit      6
9               educ      7
22  entrepreneurship      8
7        environment      9
69            health     10
3            histori     11
4             machin     12
31            person     13
2          psycholog     14
0            quantum     15
20             robot     16
18          sociolog     17
8              space     18
14               web     19


In [20]:
# step 3
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['question_text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)
print(len(train_texts), len(val_texts))

160 40


In [21]:
# step 4
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

def tokenize(texts, max_length=128):
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize(train_texts)
val_encodings = tokenize(val_texts)

In [22]:
# step 5
import torch

class ElectraDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ElectraDataset(train_encodings, train_labels)
val_dataset = ElectraDataset(val_encodings, val_labels)

In [23]:
# step 6
from transformers import ElectraForSequenceClassification, Trainer, TrainingArguments

model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='./N-Results/electra',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./N-Results/logs',
    logging_steps=10,
    # evaluation_strategy='epoch',
    # save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,3.010200
20,3.024300
30,2.986800
40,3.006500
50,2.974100
60,3.000200
70,2.981000
80,2.988200
90,2.958300
100,2.984200


TrainOutput(global_step=200, training_loss=2.97373685836792, metrics={'train_runtime': 2726.8904, 'train_samples_per_second': 0.587, 'train_steps_per_second': 0.073, 'total_flos': 105261431193600.0, 'train_loss': 2.97373685836792, 'epoch': 10.0})

In [24]:
# step 7
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Get predictions
preds = trainer.predict(val_dataset)
y_true = val_labels.values
y_pred = preds.predictions.argmax(axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=le.classes_))

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy: 0.125
Precision: 0.060555555555555564
Recall: 0.125
F1-Score: 0.0725

Classification Report:
                   precision    recall  f1-score   support

        artifici       0.40      0.67      0.50         3
      blockchain       0.00      0.00      0.00         3
          career       0.50      0.50      0.50         2
          climat       0.00      0.00      0.00         2
      cybersecur       0.00      0.00      0.00         2
            data       0.00      0.00      0.00         3
           digit       0.00      0.00      0.00         3
            educ       0.00      0.00      0.00         1
entrepreneurship       0.00      0.00      0.00         2
     environment       0.00      0.00      0.00         2
          health       0.00      0.00      0.00         1
         histori       0.00      0.00      0.00         1
          machin       0.00      0.00      0.00         1
          person       0.07      0.67      0.13         3
       psycholog       0.

C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Meet\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

In [26]:
test_texts = ["marriag best film dream wish"]  # Example test text
test_encodings = tokenize(test_texts)
model.eval()
with torch.no_grad():
    outputs = model(**{k: v for k, v in test_encodings.items()})
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = probs.argmax(dim=1).item()
    pred_topic = le.inverse_transform([pred_label])[0]
    print("Predicted topic:", pred_topic)

Predicted topic: career
